In [1]:
import pandas as pd
import json
import ast
import nltk

### file to analyze = steam_games 

In [2]:
# list 'output_steam_games.json'
rows = []
with open ('data/steam_games.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        rows.append(json.loads(line))

# save to df
df_steam_games = pd.DataFrame(rows)

In [3]:
df_steam_games.dropna(how='all', inplace=True)

In [4]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 3.4+ MB


In [5]:
df_steam_games.isna().sum()

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

In [6]:
# fill na values of 'title' with 'app_name' values
df_steam_games['title'].fillna(df_steam_games['app_name'], inplace=True)

In [7]:
# find missing id and app_name
missing = df_steam_games[df_steam_games[['id', 'app_name']].isna().any(axis=1)]
missing

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88384,NaN,NaN,NaN,NaN,http://store.steampowered.com/,NaN,NaN,NaN,NaN,19.99,False,NaN,NaN
90890,NaN,"[Action, Indie]",NaN,NaN,http://store.steampowered.com/app/317160/_/,2014-08-26,"[Action, Indie]",http://steamcommunity.com/app/317160/reviews/?...,"[Single-player, Game demo]",NaN,False,317160,NaN
119271,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [8]:
# Extract the id of the URL for Batman
df_steam_games.loc[
    df_steam_games['title'] == 'Batman: Arkham City - Game of the Year Edition', 'id'
    ] = df_steam_games['url'].apply(lambda x: x[-6:])


In [9]:
# Delete 
df_steam_games = df_steam_games[df_steam_games['url'] != 'http://store.steampowered.com/']
df_steam_games = df_steam_games[df_steam_games['id'] != '317160']

In [10]:
# checking missing again
missing = df_steam_games[df_steam_games[['id', 'app_name']].isna().any(axis=1)]
missing

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer


In [16]:
# Check content for some columns, select genres to work with
print(df_steam_games['genres'])

88310         [Action, Casual, Indie, Simulation, Strategy]
88311                  [Free to Play, Indie, RPG, Strategy]
88312     [Casual, Free to Play, Indie, Simulation, Sports]
88313                           [Action, Adventure, Casual]
88314                                                   NaN
                                ...                        
120440                [Casual, Indie, Simulation, Strategy]
120441                            [Casual, Indie, Strategy]
120442                          [Indie, Racing, Simulation]
120443                                      [Casual, Indie]
120444                                                  NaN
Name: genres, Length: 32133, dtype: object


In [11]:
# drop the columns that dont have importance to the model 
df_steam_games = df_steam_games.drop(['publisher', 'app_name', 'url', 'tags', 'reviews_url', 'specs', 'price', 'early_access', 'developer'], axis=1)

In [13]:
df_steam_games.isna().sum()

genres          3282
title              0
release_date    2066
id                 0
dtype: int64

In [15]:
# Encuentra las filas con NaN en 'release_date'
missing_release_date = df_steam_games[df_steam_games['release_date'].isna()]

# Muestra las filas con NaN en 'release_date'
missing_release_date

,genres,title,release_date,id
88314,NaN,Log Challenge,NaN,773570
88321,NaN,Icarus Six Sixty Six,NaN,724910
88329,NaN,After Life VR,NaN,772590
88330,NaN,Kitty Hawk,NaN,640250
88332,NaN,Mortars VR,NaN,711440
...,...,...,...,...
120387,NaN,SpaceWalker,NaN,705860
120394,"[Action, Casual, Indie]",BOMBFEST,NaN,729640
120395,NaN,LIV Client,NaN,755540
120398,"[Action, Adventure, Indie, Early Access]",RECHARGE COMPLETE,NaN,708070


In [ ]:
# quisiera poblar los na de genres y release_date con la api de steam

In [9]:
# extract the year of the release_date
df_steam_games['release_date'] = df_steam_games['release_date'].str.extract(r'(\d{4})')

In [10]:
# get dummies of genres 
df_steam_games['genres'] = df_steam_games['genres'].fillna('[]')  # Rellenar los valores faltantes con una lista vacía
df_steam_games['genres'] = df_steam_games['genres'].apply(lambda x: ', '.join(x))  # Convertir la lista de géneros a una cadena separada por comas

# create columns
dummy_genres = df_steam_games['genres'].str.get_dummies(', ')

# Concat
df_steam_games = pd.concat([df_steam_games, dummy_genres], axis=1)

In [11]:
df_steam_games

,genres,title,release_date,id,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,...,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,[,]
88310,"Action, Casual, Indie, Simulation, Strategy",Lost Summoner Kitty,2018,761140,0,1,0,0,0,1,...,0,1,0,0,1,0,0,0,0,0
88311,"Free to Play, Indie, RPG, Strategy",Ironbound,2018,643980,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
88312,"Casual, Free to Play, Indie, Simulation, Sports",Real Pool 3D - Poolians,2017,670290,0,0,0,0,0,1,...,0,1,0,1,0,0,0,0,0,0
88313,"Action, Adventure, Casual",弹炸人2222,2017,767400,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
88314,"[, ]",NaN,NaN,773570,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,"Casual, Indie, Simulation, Strategy",Colony On Mars,2018,773640,0,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,0
120441,"Casual, Indie, Strategy",LOGistICAL: South Africa,2018,733530,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
120442,"Indie, Racing, Simulation",Russian Roads,2018,610660,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
120443,"Casual, Indie",EXIT 2 - Directions,2017,658870,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# borro  las columnas que no  necesito

df_steam_games = df_steam_games.iloc[:, :-1]
# normalizamos el nombre de algunas columnas
df_steam_games.rename(columns={'Animation &amp; Modeling': 'Animation and Modeling'}, inplace=True)
df_steam_games.rename(columns={'Design &amp; Illustration': 'Design and Illustration'}, inplace=True)
df_steam_games.rename(columns={'id': 'item_id'}, inplace=True)

In [ ]:
df_steam_games

In [ ]:
df_steam_games.drop('genres' , axis = 1, inplace = True)

In [ ]:
df_steam_games.loc[df_steam_games['item_id'] == '']

In [ ]:
df_steam_games.to_csv('df_steaam_games.csv', index=False)